## Model Predictive Control Example

To determine the optimal control input $u \in \mathbb{R}^m$ applied to the linear time-invariant (LTI) system $\left( A, B\right) \in \left( \mathbb{R}^{n \times n}, \mathbb{R}^{n \times m} \right)$ with state $x \in \mathbb{R}^n$, we solve the following optimization problem:

\begin{align}
\min_{\{x^i, u^i\}}. \quad & \Vert x^N \Vert_P^2 + \sum_{i=0}^{N-1} \Vert x^i \Vert_Q^2 + \Vert u^i \Vert_R^2\\
\text{s.t.} \quad &x^{i+1} = A x^i + B u^i \quad \forall i \in \{0, ..., N-1\} \\
& \Vert u^i \Vert_\infty \leq u^\mathrm{max} \quad \forall i \in \{0, ..., N-1\} \\
\end{align}

The prediction horizon is $N$, the control input is constrained within a box of size $u^\mathrm{max}$ and the cost matrices are positive definite: $P, Q, R \in \mathbb{S}_{++}^n, \mathbb{S}_{++}^n, \mathbb{S}_{++}^m$. Usually, $P$ is chosen as the solution to the discrete-time algebraic Riccati equation for the given LTI system and cost matrices $Q, R$.

Let's define the corresponding CVXPY problem:

In [1]:
import cvxpy as cp

# define dimensions
N, n, m = 10, 3, 2

# define variables
u = []
x = []
for i in range(N-1):
    u.append(cp.Variable(m, name='u_%d' % i))
    x.append(cp.Variable(n, name='x_%d' % i))
x.append(cp.Variable(n, name='x_%d' % N))

# define parameters
P = cp.Parameter((n, n), PSD=True, name='P')
Q = cp.Parameter((n, n), PSD=True, name='Q')
R = cp.Parameter((m, m), PSD=True, name='R')
A = cp.Parameter((n, n), name='A')
B = cp.Parameter((n, m), name='B')
u_max = cp.Parameter(name='u_max')

# define objective
obj_func = cp.quad_form(x[len(x)-1], P)
for i in range(N-1):
    obj_func += (cp.quad_form(x[i], Q) + cp.quad_form(u[i], R))
objective = cp.Minimize(obj_func)

# define constraints
constraints = []
for i in range(N-1):
    constraints.append(x[i+1] == A@x[i] + B@u[i])
    constraints.append(cp.norm(u[i], 'inf') <= u_max)

# define problem
problem = cp.Problem(objective, constraints)

Assign parameter values and solve the problem.

In [ ]:
# TODO

val = problem.solve()

Generating C source for the problem is as easy as:

In [ ]:
import sys
sys.path.append('../')
import cvxpygen as cpg

cpg.generate_code(problem, code_dir='MPC_code')

Now, you can use a python wrapper around the generated code as a custom CVXPY solve method:

In [ ]:
from MPC_code.cpg_solver import cpg_solve
import numpy as np
import pickle
import time

# load the serialized problem formulation
with open('MPC_code/problem.pickle', 'rb') as f:
    prob = pickle.load(f)

# assign parameter values
# TODO

# solve problem conventionally
t0 = time.time()
# CVXPY chooses eps_abs=eps_rel=1e-5, max_iter=10000, polish=True by default,
# however, we choose the OSQP default values here, as they are used for code generation as well
val = prob.solve(eps_abs=1e-3, eps_rel=1e-3, max_iter=4000, polish=False)
t1 = time.time()
print('\nPython solve time:', 1000*(t1-t0), 'ms')
print('Python objective function value:', val)

# solve problem with C code via python wrapper
prob.register_solve('CPG', cpg_solve)
t0 = time.time()
val = prob.solve(method='CPG')
t1 = time.time()
print('\nC solve time:', 1000*(t1-t0), 'ms')
print('C objective function value:', val)